## Churn Prediction

#### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### Load DataSet

In [2]:
df = pd.read_csv('/home/rguktrkvalley/Desktop/churn_prediction/Churn_Modelling.csv')
X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

### Encoding the categorical Labels

In [3]:
from sklearn.preprocessing import LabelEncoder
LeX=LabelEncoder()
X[:,2]=LeX.fit_transform(X[:,2])

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])], remainder='passthrough')
X=np.array(ct.fit_transform(X))

#### Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)

#### Spliting the dataset into train and test data

In [8]:
#Splitting dataset into train and test set
from sklearn.model_selection import train_test_split
#Set any int as random state for reprodicibility
X_train, X_test, Y_train, Y_test=train_test_split(X,y,test_size=0.2,random_state=42)


#### Import model and layers

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2025-07-10 10:02:08.780292: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-10 10:02:09.023586: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-10 10:02:09.024840: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-10 10:02:14.054321: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
input_dim=X_train.shape[1]

#### Inputing layers into model

In [14]:
model=Sequential([Dense(64,activation='relu',input_shape=(input_dim,)),    #Adding first hidden layer
                  Dense(32,activation='relu'),                            #Second hidden layer
                  Dense(1,activation='sigmoid')]) 

#### Compiling the model

In [15]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Training the model

In [16]:
history=model.fit(X_train,Y_train,batch_size=32, epochs=55,validation_split=0.2)

Epoch 1/55
200/200 [==============================] - 2s 5ms/step - loss: 0.4696 - accuracy: 0.7895 - val_loss: 0.4047 - val_accuracy: 0.8381
Epoch 2/55
200/200 [==============================] - 1s 4ms/step - loss: 0.3882 - accuracy: 0.8428 - val_loss: 0.3696 - val_accuracy: 0.8500
Epoch 3/55
200/200 [==============================] - 1s 3ms/step - loss: 0.3551 - accuracy: 0.8555 - val_loss: 0.3544 - val_accuracy: 0.8581
Epoch 4/55
200/200 [==============================] - 1s 4ms/step - loss: 0.3437 - accuracy: 0.8591 - val_loss: 0.3557 - val_accuracy: 0.8544
Epoch 5/55
200/200 [==============================] - 1s 3ms/step - loss: 0.3391 - accuracy: 0.8606 - val_loss: 0.3463 - val_accuracy: 0.8519
Epoch 6/55
200/200 [==============================] - 1s 3ms/step - loss: 0.3347 - accuracy: 0.8594 - val_loss: 0.3476 - val_accuracy: 0.8544
Epoch 7/55
200/200 [==============================] - 1s 3ms/step - loss: 0.3327 - accuracy: 0.8625 - val_loss: 0.3507 - val_accuracy: 0.8581
Epoch 

#### Evaluating The model

In [17]:
loss,accuracy=model.evaluate(X_test,Y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")

63/63 [==============================] - 0s 3ms/step - loss: 0.3949 - accuracy: 0.8380
Test Loss: 0.3949
Accuracy: 0.8380


#### Single Prediction and test data prediction

In [21]:
print(model.predict(sc.transform([[1.0, 0.0, 0.0, 1, 447, 31, 7, 0.0, 4, 1,1, 519360]]))>0.5)
print(model.predict(sc.transform([[0.0,1.0,0.0, 0, 315, 25, 3, 12344, 2, 1,0, 12425]]))>0.5)
predictions=model.predict(X_test)

1/1 [==============================] - 0s 27ms/step
[[False]]
1/1 [==============================] - 0s 30ms/step
[[False]]
63/63 [==============================] - 0s 2ms/step


In [23]:
Y_pred=model.predict(X_test)
Y_pred=(Y_pred>0.5)
print(np.concatenate((Y_pred.reshape(len(Y_pred),1),Y_test.reshape(len(Y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [1 1]
 [0 1]
 [0 1]]


#### Confusion metrix and accuracy

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(Y_test,Y_pred)
print(cm)
accuracy_score(Y_test,Y_pred)

[[1432  175]
 [ 149  244]]


0.838

#### Saving The model

In [27]:

# AFTER training is completed (after model.fit)
from tensorflow.keras.models import load_model
import joblib

# ✅ Save the model
model.save("model.keras")  # or "model.h5"
model.save("model.h5")
# ✅ Save the scaler
joblib.dump(sc, "scaler.pkl")


/home/rguktrkvalley/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['scaler.pkl']